# Notebook 08: Model Monitoring & CloudWatch Dashboard
- we will be using the 20% split production data to simulate drift monitoring

## Setup

In [22]:
import os
import sys
import json
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import boto3
import sagemaker
from sagemaker.session import Session

sys.path.append("..")
from config.config import (
    BUCKET_NAME,
    S3_PREFIX,
    AWS_REGION,
)

print(f"SageMaker SDK version: {sagemaker.__version__}")

SageMaker SDK version: 2.256.0


In [23]:
# Initialize sessions and clients
boto_session = boto3.Session(region_name=AWS_REGION)
sagemaker_session = Session(boto_session=boto_session)

sm_client = boto3.client('sagemaker', region_name=AWS_REGION)
s3_client = boto3.client('s3', region_name=AWS_REGION)
cw_client = boto3.client('cloudwatch', region_name=AWS_REGION)

# Get execution role
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam_client = boto3.client('iam')
    role = iam_client.get_role(RoleName='sagemaker-execution-role')['Role']['Arn']

print(f"Region: {AWS_REGION}")
print(f"Role: {role}")
print(f"Bucket: {BUCKET_NAME}")

Couldn't call 'get_role' to get Role ARN from role name fatimat-admin to get Role path.


Region: us-east-1
Role: arn:aws:iam::306617143793:role/sagemaker-execution-role
Bucket: nfci-forecasting-306617143793


## Get the Registered Model from notebook 7


In [24]:
# Get the latest registered model
model_package_group = "nfci-forecasting-models"

response = sm_client.list_model_packages(
    ModelPackageGroupName=model_package_group,
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=1
)

latest_model = response['ModelPackageSummaryList'][0]
model_package_arn = latest_model['ModelPackageArn']

print(f"Model Package ARN: {model_package_arn}")
print(f"Version: {latest_model['ModelPackageVersion']}")
print(f"Status: {latest_model['ModelApprovalStatus']}")

Model Package ARN: arn:aws:sagemaker:us-east-1:306617143793:model-package/nfci-forecasting-models/2
Version: 2
Status: Approved


In [25]:
# Get model details
model_details = sm_client.describe_model_package(ModelPackageName=model_package_arn)

model_data_url = model_details['InferenceSpecification']['Containers'][0]['ModelDataUrl']
image_uri = model_details['InferenceSpecification']['Containers'][0]['Image']

print(f"Model artifact: {model_data_url}")
print(f"Container image: {image_uri}")

Model artifact: s3://nfci-forecasting-306617143793/models/artifacts/pipelines-87qaih00spbr-TrainDeepAR-LtBpVP1FkF/output/model.tar.gz
Container image: 522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1


## Deploy Endpoint with Data Capture

Data Capture logs all inference requests and responses to S3.
This data is used by Model Monitor to detect drift.

In [26]:
from sagemaker.model_monitor import DataCaptureConfig

# Configure data capture
data_capture_prefix = f"{S3_PREFIX['models']}/data-capture"

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,  # Capture all requests 
    destination_s3_uri=f"s3://{BUCKET_NAME}/{data_capture_prefix}",
    capture_options=["Input", "Output"],
    csv_content_types=["text/csv"],
    json_content_types=["application/json", "application/jsonlines"],
)

print(f"Data capture destination: s3://{BUCKET_NAME}/{data_capture_prefix}")

Data capture destination: s3://nfci-forecasting-306617143793/models/artifacts/data-capture


In [27]:
from sagemaker.model import ModelPackage

# Create model from registered package
endpoint_name = "nfci-forecasting-endpoint"

model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

print(f"Deploying endpoint: {endpoint_name}")


Deploying endpoint: nfci-forecasting-endpoint


In [28]:
# Deploy the endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

print(f"Endpoint deployed: {endpoint_name}")

---------!Endpoint deployed: nfci-forecasting-endpoint


## Load Production Data

In Notebook 04, we held out 20% of the data for production evaluation.


In [29]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import pandas as pd
import io

# Create predictor from endpoint
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

print(f"Predictor configured for endpoint: {endpoint_name}")

Predictor configured for endpoint: nfci-forecasting-endpoint


In [30]:
# Load the production holdout data (last 20% of dates)
obj = s3_client.get_object(
    Bucket=BUCKET_NAME,
    Key='data/splits/production/features.parquet'
)
production_df = pd.read_parquet(io.BytesIO(obj['Body'].read()))

print(f"Production data shape: {production_df.shape}")
print(f"Date range: {production_df['date'].min()} to {production_df['date'].max()}")
print(f"States: {production_df['state_fips'].nunique()}")
print(f"\nColumns: {list(production_df.columns)[:10]}...")

Production data shape: (2400, 80)
Date range: 2020-01-01 00:00:00 to 2023-12-01 00:00:00
States: 50

Columns: ['state_fips', 'date', 'UNRATE', 'PAYEMS', 'CIVPART', 'EMRATIO', 'U6RATE', 'AWHMAN', 'AHETPI', 'CPIAUCSL']...


In [31]:
# Get actual NFCI values from production period
# These are the ground truth values to compare against prediction
actual_nfci = production_df.groupby('date')['NFCI'].first().sort_index()

print(f"Production period NFCI values: {len(actual_nfci)} months")
print(f"\nFirst 5 values:")
print(actual_nfci.head())

Production period NFCI values: 48 months

First 5 values:
date
2020-01-01   -0.61879
2020-02-01   -0.29288
2020-03-01    0.28082
2020-04-01    0.16847
2020-05-01   -0.26647
Name: NFCI, dtype: float64


In [33]:
# Load the inference records (from pipeline preprocessing)
# These contain the history needed for DeepAR to make predictions
obj = s3_client.get_object(
    Bucket=BUCKET_NAME,
    Key='pipeline/preprocess/inference/inference.json'
)
content = obj['Body'].read().decode('utf-8')
inference_records = [json.loads(line) for line in content.strip().split('\n')]

print(f"Loaded {len(inference_records)} inference records (one per state)")
print(f"Each record has {len(inference_records[0]['target'])} months of history")

Loaded 50 inference records (one per state)
Each record has 180 months of history



## Generate Predictions and Compare to Actuals
send inference requests and compare predictions to actual NFCI values from the production holdout period.

In [34]:
# Send predictions for multiple states
print("Sending predictions for production period...\n")

all_predictions = []
num_states_to_test = 10  # Test with 10 states

for i in range(num_states_to_test):
    record = inference_records[i]
    
    # DeepAR expects instances and configuration
    payload = {
        "instances": [record],
        "configuration": {
            "num_samples": 100,
            "output_types": ["mean", "quantiles"],
            "quantiles": ["0.1", "0.5", "0.9"]
        }
    }
    
    response = predictor.predict(payload)
    mean_forecast = response['predictions'][0]['mean']
    all_predictions.append(mean_forecast)
    
    print(f"  State {i+1}: forecast = {mean_forecast[:3]}... (12 values)")
    time.sleep(0.5)

print(f"\nGenerated predictions for {num_states_to_test} states")

Sending predictions for production period...

  State 1: forecast = [-0.3190772533, -0.2437551171, -0.4000681937]... (12 values)
  State 2: forecast = [-0.3485395908, -0.2797142863, -0.4239366651]... (12 values)
  State 3: forecast = [-0.3533447385, -0.1810630858, -0.3974384367]... (12 values)
  State 4: forecast = [-0.3242557943, -0.1545181721, -0.3737348616]... (12 values)
  State 5: forecast = [-0.2754732072, -0.2501436472, -0.3109013736]... (12 values)
  State 6: forecast = [-0.3175070286, -0.2981271148, -0.3450265825]... (12 values)
  State 7: forecast = [-0.3254566193, -0.4094000161, -0.4434669018]... (12 values)
  State 8: forecast = [-0.2798507214, -0.3669470251, -0.4023587108]... (12 values)
  State 9: forecast = [-0.3438080251, -0.3579233885, -0.4175674915]... (12 values)
  State 10: forecast = [-0.3915121555, -0.391107738, -0.4414982498]... (12 values)

Generated predictions for 10 states


## Actual Vs Forcast

In [35]:
import numpy as np

# Average predictions across states (NFCI is a national index)
avg_prediction = np.mean(all_predictions, axis=0)

# Get actual values for the forecast period (first 12 months of production)
actual_values = actual_nfci.values[:12]

# Compute metrics
rmse = np.sqrt(np.mean((actual_values - avg_prediction) ** 2))
mae = np.mean(np.abs(actual_values - avg_prediction))

print("Production Period Evaluation:")
print("=" * 40)
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"\nForecast vs Actual (first 6 months):")
print(f"{'Month':<8} {'Predicted':<12} {'Actual':<12} {'Error':<12}")
print("-" * 44)
for i in range(6):
    error = avg_prediction[i] - actual_values[i]
    print(f"{i+1:<8} {avg_prediction[i]:<12.4f} {actual_values[i]:<12.4f} {error:<12.4f}")

Production Period Evaluation:
RMSE: 0.3070
MAE:  0.2459

Forecast vs Actual (first 6 months):
Month    Predicted    Actual       Error       
--------------------------------------------
1        -0.3279      -0.6188      0.2909      
2        -0.2933      -0.2929      -0.0004     
3        -0.3956      0.2808       -0.6764     
4        -0.3718      0.1685       -0.5403     
5        -0.3754      -0.2665      -0.1089     
6        -0.2925      -0.3839      0.0914      


In [36]:
# Wait for data capture to sync to S3
print("Waiting 60 seconds for data capture to sync to S3...")
time.sleep(60)

Waiting 60 seconds for data capture to sync to S3...


In [37]:
# Check captured data
response = s3_client.list_objects_v2(
    Bucket=BUCKET_NAME,
    Prefix=data_capture_prefix,
    MaxKeys=10
)

print(f"\nCaptured data files:")
for obj in response.get('Contents', []):
    print(f"  {obj['Key']} ({obj['Size']} bytes)")


Captured data files:
  models/artifacts/data-capture/nfci-forecasting-endpoint/AllTraffic/2026/02/07/03/56-42-794-e109eb01-62c6-463a-8d60-ba1640672f58.jsonl (86427 bytes)
  models/artifacts/data-capture/nfci-forecasting-endpoint/AllTraffic/2026/02/07/03/59-57-770-9000860c-1c54-450c-b221-ffae7ac8db6e.jsonl (49880 bytes)
  models/artifacts/data-capture/nfci-forecasting-endpoint/AllTraffic/2026/02/07/04/19-36-036-fb12aa05-394d-481c-b048-b147890037cc.jsonl (173068 bytes)


## Create Baseline for Model Monitor

The baseline captures statistics from training data.
Model Monitor compares live data against this baseline to detect drift.


In [38]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

# Create Model Monitor instance
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

print("Model Monitor instance created")

Model Monitor instance created


## CloudWatch Metrics

- SageMaker automatically publishes endpoint metrics to CloudWatch.
- We can also publish custom metrics for NFCI forecasts.

In [39]:
# Check built-in SageMaker endpoint metrics
print("Built-in SageMaker Endpoint Metrics:")

metrics_to_check = [
    'Invocations',
    'InvocationsPerInstance', 
    'ModelLatency',
    'OverheadLatency',
    'Invocation4XXErrors',
    'Invocation5XXErrors',
]

for metric_name in metrics_to_check:
    print(f"  AWS/SageMaker: {metric_name}")

Built-in SageMaker Endpoint Metrics:
  AWS/SageMaker: Invocations
  AWS/SageMaker: InvocationsPerInstance
  AWS/SageMaker: ModelLatency
  AWS/SageMaker: OverheadLatency
  AWS/SageMaker: Invocation4XXErrors
  AWS/SageMaker: Invocation5XXErrors


In [40]:
# Publish custom metrics for NFCI forecasting
def publish_forecast_metrics(forecast_mean, actual_value=None):
    """
    Publish custom CloudWatch metrics for NFCI forecasting.
    """
    timestamp = datetime.utcnow()
    
    metrics = [
        {
            'MetricName': 'ForecastMean',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Timestamp': timestamp,
            'Value': float(forecast_mean[0]),  # First month forecast
            'Unit': 'None'
        },
    ]
    
    # If actuals exists, compute and publish error
    if actual_value is not None:
        error = abs(forecast_mean[0] - actual_value)
        metrics.append({
            'MetricName': 'ForecastError',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Timestamp': timestamp,
            'Value': error,
            'Unit': 'None'
        })
    
    cw_client.put_metric_data(
        Namespace='NFCI-Forecasting',
        MetricData=metrics
    )
    
    return metrics

print("Custom metric function defined")

Custom metric function defined


In [41]:
# Send predictions and publish metrics with actual values
print("Publishing production metrics to CloudWatch...\n")

# Use the predictions and actuals we already computed
for i in range(min(12, len(actual_values))):
    # Publish forecast and actual comparison
    metrics = publish_forecast_metrics(
        forecast_mean=[avg_prediction[i]],
        actual_value=actual_values[i]
    )
    
    error = abs(avg_prediction[i] - actual_values[i])
    print(f"Month {i+1}: Forecast={avg_prediction[i]:.4f}, Actual={actual_values[i]:.4f}, Error={error:.4f}")
    time.sleep(1)

# Also publish summary metrics
cw_client.put_metric_data(
    Namespace='NFCI-Forecasting',
    MetricData=[
        {
            'MetricName': 'ProductionRMSE',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Value': rmse,
            'Unit': 'None'
        },
        {
            'MetricName': 'ProductionMAE',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'Model', 'Value': 'DeepAR'},
            ],
            'Value': mae,
            'Unit': 'None'
        },
    ]
)

print(f"\nPublished ProductionRMSE={rmse:.4f}, ProductionMAE={mae:.4f}")

Publishing production metrics to CloudWatch...

Month 1: Forecast=-0.3279, Actual=-0.6188, Error=0.2909
Month 2: Forecast=-0.2933, Actual=-0.2929, Error=0.0004
Month 3: Forecast=-0.3956, Actual=0.2808, Error=0.6764
Month 4: Forecast=-0.3718, Actual=0.1685, Error=0.5403
Month 5: Forecast=-0.3754, Actual=-0.2665, Error=0.1089
Month 6: Forecast=-0.2925, Actual=-0.3839, Error=0.0914
Month 7: Forecast=-0.2792, Actual=-0.4760, Error=0.1967
Month 8: Forecast=-0.3220, Actual=-0.5085, Error=0.1866
Month 9: Forecast=-0.3164, Actual=-0.4918, Error=0.1754
Month 10: Forecast=-0.3350, Actual=-0.5052, Error=0.1702
Month 11: Forecast=-0.3863, Actual=-0.5669, Error=0.1806
Month 12: Forecast=-0.2706, Actual=-0.6038, Error=0.3332

Published ProductionRMSE=0.3070, ProductionMAE=0.2459


## Create CloudWatch Alarm

Alarms notify when metrics exceed thresholds.

In [42]:
# Create alarm for high latency
cw_client.put_metric_alarm(
    AlarmName='NFCI-Endpoint-HighLatency',
    ComparisonOperator='GreaterThanThreshold',
    EvaluationPeriods=2,
    MetricName='ModelLatency',
    Namespace='AWS/SageMaker',
    Period=300,  # 5 minutes
    Statistic='Average',
    Threshold=5000000,  # 5 seconds in microseconds
    ActionsEnabled=False,  # Set to True and add SNS ARN for notifications
    AlarmDescription='Alarm when model latency exceeds 5 seconds',
    Dimensions=[
        {'Name': 'EndpointName', 'Value': endpoint_name},
        {'Name': 'VariantName', 'Value': 'AllTraffic'},
    ],
)

print("Created alarm: NFCI-Endpoint-HighLatency")

Created alarm: NFCI-Endpoint-HighLatency


In [43]:
# Create alarm for invocation errors
cw_client.put_metric_alarm(
    AlarmName='NFCI-Endpoint-Errors',
    ComparisonOperator='GreaterThanThreshold',
    EvaluationPeriods=1,
    MetricName='Invocation5XXErrors',
    Namespace='AWS/SageMaker',
    Period=300,
    Statistic='Sum',
    Threshold=5,
    ActionsEnabled=False,
    AlarmDescription='Alarm when 5XX errors exceed 5 in 5 minutes',
    Dimensions=[
        {'Name': 'EndpointName', 'Value': endpoint_name},
        {'Name': 'VariantName', 'Value': 'AllTraffic'},
    ],
)

print("Created alarm: NFCI-Endpoint-Errors")

Created alarm: NFCI-Endpoint-Errors


In [44]:
# List all alarms
response = cw_client.describe_alarms(
    AlarmNamePrefix='NFCI-'
)

print("CloudWatch Alarms:")
print("-" * 50)
for alarm in response['MetricAlarms']:
    print(f"  {alarm['AlarmName']}: {alarm['StateValue']}")

CloudWatch Alarms:
--------------------------------------------------
  NFCI-Endpoint-Errors: INSUFFICIENT_DATA
  NFCI-Endpoint-HighLatency: INSUFFICIENT_DATA


- alarm output above is normal because no metric history yet 

## Create CloudWatch Dashboard

A dashboard provides a single view of all monitoring metrics.

In [45]:
# Define dashboard layout
dashboard_name = "NFCI-Forecasting-Dashboard"

dashboard_body = {
    "widgets": [
        # Row 1: Title
        {
            "type": "text",
            "x": 0, "y": 0,
            "width": 24, "height": 1,
            "properties": {
                "markdown": "# NFCI Forecasting - Model Monitoring Dashboard"
            }
        },
        # Row 2: Endpoint Invocations
        {
            "type": "metric",
            "x": 0, "y": 1,
            "width": 8, "height": 6,
            "properties": {
                "title": "Endpoint Invocations",
                "metrics": [
                    ["AWS/SageMaker", "Invocations", "EndpointName", endpoint_name, "VariantName", "AllTraffic"]
                ],
                "period": 60,
                "stat": "Sum",
                "region": AWS_REGION,
                "view": "timeSeries"
            }
        },
        # Row 2: Model Latency
        {
            "type": "metric",
            "x": 8, "y": 1,
            "width": 8, "height": 6,
            "properties": {
                "title": "Model Latency (ms)",
                "metrics": [
                    ["AWS/SageMaker", "ModelLatency", "EndpointName", endpoint_name, "VariantName", "AllTraffic"]
                ],
                "period": 60,
                "stat": "Average",
                "region": AWS_REGION,
                "view": "timeSeries",
                "yAxis": {"left": {"label": "Microseconds"}}
            }
        },
        # Row 2: Errors
        {
            "type": "metric",
            "x": 16, "y": 1,
            "width": 8, "height": 6,
            "properties": {
                "title": "Invocation Errors",
                "metrics": [
                    ["AWS/SageMaker", "Invocation4XXErrors", "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
                    ["AWS/SageMaker", "Invocation5XXErrors", "EndpointName", endpoint_name, "VariantName", "AllTraffic"]
                ],
                "period": 60,
                "stat": "Sum",
                "region": AWS_REGION,
                "view": "timeSeries"
            }
        },
        # Row 3: Custom NFCI Forecast Metrics
        {
            "type": "metric",
            "x": 0, "y": 7,
            "width": 8, "height": 6,
            "properties": {
                "title": "NFCI Forecast vs Actual",
                "metrics": [
                    ["NFCI-Forecasting", "ForecastMean", "EndpointName", endpoint_name, "Model", "DeepAR"],
                    ["NFCI-Forecasting", "ForecastError", "EndpointName", endpoint_name, "Model", "DeepAR"]
                ],
                "period": 60,
                "stat": "Average",
                "region": AWS_REGION,
                "view": "timeSeries"
            }
        },
        # Row 3: Production Metrics (RMSE, MAE)
        {
            "type": "metric",
            "x": 8, "y": 7,
            "width": 8, "height": 6,
            "properties": {
                "title": "Production Accuracy",
                "metrics": [
                    ["NFCI-Forecasting", "ProductionRMSE", "EndpointName", endpoint_name, "Model", "DeepAR"],
                    ["NFCI-Forecasting", "ProductionMAE", "EndpointName", endpoint_name, "Model", "DeepAR"]
                ],
                "period": 300,
                "stat": "Average",
                "region": AWS_REGION,
                "view": "singleValue"
            }
        },
        # Row 3: Alarm Status
        {
            "type": "alarm",
            "x": 16, "y": 7,
            "width": 8, "height": 6,
            "properties": {
                "title": "Alarm Status",
                "alarms": [
                    f"arn:aws:cloudwatch:{AWS_REGION}:{boto3.client('sts').get_caller_identity()['Account']}:alarm:NFCI-Endpoint-HighLatency",
                    f"arn:aws:cloudwatch:{AWS_REGION}:{boto3.client('sts').get_caller_identity()['Account']}:alarm:NFCI-Endpoint-Errors"
                ]
            }
        },
    ]
}

print("Dashboard layout defined")

Dashboard layout defined


### Create CW dashboard

In [46]:
# Create the dashboard
cw_client.put_dashboard(
    DashboardName=dashboard_name,
    DashboardBody=json.dumps(dashboard_body)
)

dashboard_url = f"https://{AWS_REGION}.console.aws.amazon.com/cloudwatch/home?region={AWS_REGION}#dashboards:name={dashboard_name}"

print(f"Dashboard created: {dashboard_name}")
print(f"\nView dashboard at:")
print(dashboard_url)

Dashboard created: NFCI-Forecasting-Dashboard

View dashboard at:
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#dashboards:name=NFCI-Forecasting-Dashboard


## View Monitoring Summary

In [47]:
# Get endpoint status
endpoint_desc = sm_client.describe_endpoint(EndpointName=endpoint_name)

print("MONITORING SUMMARY")
print("=" * 60)
print(f"\nEndpoint: {endpoint_name}")
print(f"Status: {endpoint_desc['EndpointStatus']}")
print(f"Instance: {endpoint_desc['ProductionVariants'][0]['CurrentInstanceCount']}x ml.m5.large")
print(f"\nData Capture: s3://{BUCKET_NAME}/{data_capture_prefix}")
print(f"\nCloudWatch Dashboard: {dashboard_name}")
print(f"\nAlarms:")
print(f"  - NFCI-Endpoint-HighLatency")
print(f"  - NFCI-Endpoint-Errors")
print("=" * 60)

MONITORING SUMMARY

Endpoint: nfci-forecasting-endpoint
Status: InService
Instance: 1x ml.m5.large

Data Capture: s3://nfci-forecasting-306617143793/models/artifacts/data-capture

CloudWatch Dashboard: NFCI-Forecasting-Dashboard

Alarms:
  - NFCI-Endpoint-HighLatency
  - NFCI-Endpoint-Errors


## Cleanup

**Delete the endpoint to avoid ongoing charges.**



In [48]:
# Uncomment and run when ready to clean up

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)
print(f"eleted endpoint: {endpoint_name}")

# Delete endpoint config
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
print(f"Deleted endpoint config")

# Delete alarms
cw_client.delete_alarms(AlarmNames=['NFCI-Endpoint-HighLatency', 'NFCI-Endpoint-Errors'])
print("Deleted CloudWatch alarms")

# Delete dashboard
cw_client.delete_dashboards(DashboardNames=[dashboard_name])
print(f"Deleted dashboard: {dashboard_name}")

print("\nCleanup complete!")

eleted endpoint: nfci-forecasting-endpoint
Deleted endpoint config
Deleted CloudWatch alarms
Deleted dashboard: NFCI-Forecasting-Dashboard

Cleanup complete!
